# Download salonecriture.org website to json

In [1]:
import cgi
import json
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import os

## 1. Testing:

In [2]:
html=requests.get('https://www.salonecriture.org/actualités/')
soup=BeautifulSoup(html.content, 'html.parser')
soup=soup.find('div', attrs={'class': 'jtpl-content__inner content-options cc-content-parent'})

### Links

In [3]:
all_links = []
for ref in soup.findAll('a', attrs={'class': 'blogreadmore'}):
    all_links.append(ref['href'])

In [4]:
all_links

['/2017/03/02/application-pour-iphone-disponible/',
 '/2017/02/27/sortie-d-une-application-mobile-android-pour-le-salon/',
 '/2017/02/12/ghostwriter-écrivain-fantôme-prête-plume-qu-est-ce/',
 '/2017/01/28/secrétaire-juridique-l-écriture-ce-vecteur-codé-et-procédural/',
 '/2017/01/25/maître-marc-bonnant-à-l-honneur/',
 '/2017/01/17/harmonie-keller-matthey-blogueuse-l-écriture-est-un-art-libérateur/',
 '/2017/01/14/exposition-sur-l-alphabet-a-ka-u-ku/',
 '/2017/01/11/rédacteur-technique-ou-l-art-de-la-vulgarisation/',
 '/2017/01/07/qu-est-ce-qu-un-scriptorium/',
 '/2016/12/20/quels-étaient-les-moyens-de-fabriquer-de-l-encre-noire-au-moyen-âge/',
 '/2016/12/14/traducteur-de-l-art-de-transposer-sans-dénaturer/',
 '/2016/12/09/de-la-lettre-manuscrite-au-sms/',
 '/2016/12/04/quelle-écriture-était-utilisée-et-transmise-exclusivement-par-des-femmes/',
 '/2016/11/29/papyrologue-éditeur-de-textes-anciens/',
 '/2016/11/26/qu-est-ce-qu-un-alphabet-consonantique/',
 '/2016/11/18/quel-est-le-mythe-à

### Titles

In [5]:
for title in soup.findAll('h2', attrs={'class': 'j-blog-headline'}):
    print(title.text)

Application pour iPhone disponible
Sortie d'une application mobile (Android) pour le salon
Ghostwriter, écrivain fantôme, prête-plume... qu'est-ce?
Secrétaire juridique - L'écriture, ce vecteur codé et procédural
Maître Marc Bonnant à l'honneur
Harmonie Keller Matthey, blogueuse: "L'écriture est un art libérateur"
Exposition sur l'alphabet a ka u ku
Rédacteur technique ou l'art de la vulgarisation
Qu'est-ce qu'un scriptorium?
Quels étaient les moyens de fabriquer de l'encre noire au Moyen-Âge?
Traducteur: de l'art de transposer sans dénaturer
De la lettre manuscrite au sms
Quelle écriture était utilisée et transmise exclusivement par des femmes?
Papyrologue: éditeur de textes anciens
Qu'est-ce qu'un alphabet consonantique?
Quel est le mythe à l'origine de l'écriture chinoise?
Le correcteur au coeur des mots
Que reste-t-il des écrits gaulois?
L'expertise en écritures: différencier le vrai du faux
Comment fonctionne l'alphabet braille et comment les aveugles peuvent-ils lire à l'écran ?


### Dates

In [6]:
for date in soup.findAll('time', attrs={'class': 'datetime'}):
    print(date.text.replace('\n', ' '))

 jeu. 02 mars 2017 
 mar. 28 févr. 2017 
 dim. 12 févr. 2017 
 mer. 01 févr. 2017 
 mer. 25 janv. 2017 
 mar. 17 janv. 2017 
 sam. 14 janv. 2017 
 mer. 11 janv. 2017 
 sam. 07 janv. 2017 
 mar. 20 déc. 2016 
 mer. 14 déc. 2016 
 ven. 09 déc. 2016 
 dim. 04 déc. 2016 
 mar. 29 nov. 2016 
 sam. 26 nov. 2016 
 ven. 18 nov. 2016 
 mar. 15 nov. 2016 
 mer. 09 nov. 2016 
 mer. 02 nov. 2016 
 ven. 28 oct. 2016 


### Summary

In [7]:
for summary in soup.findAll('div', attrs={'class':'j-module n j-text '}):
    txt2 = summary.text.replace("\n   ", "")
    txt3 = txt2[1:len(txt2)-1]
    print(txt3)
    print('-----------------------------')

Dès maintenant, vous pouvez télécharger l'application mobile du Salon International de l'Écriture pour iPhone.

 Vous y trouverez les informations concernant le programme du Salon, les actualités et les informations pratiques. N'hésitez pas à la télécharger pour avoir partout avec vous le programme!
-----------------------------
Dès aujourd'hui vous trouverez le programme des activités du Salon sur notre toute nouvelle application développée par Daniel Guggenheim, étudiant en informatique à l'EPFL.

  

 Cette application permet aux visiteurs du Salon International de l’Écriture :


D’avoir sur soi en permanence le programme des différentes activités, conférences, ateliers, expositions et animations du Salon les 3 et 4 mars 2017. Les changements éventuels de dernière minute seront signalés par le biais d’une alerte en haut de l’onglet du programme. 
D’obtenir des informations pratiques concernant notamment les horaires du Salon et l’accès aux sites. 
D’être tenu au courant tout au long

## 2. Retrieve all articles
With the article, 2 json will be generated: 1 with the information about the article, the other with the content of the article.

**articles_infos:** Array of object, with each object:
```
{
  "category" : "Actualité - Organisation - Exposition", # Article category
  "date" : "2017-03-02", # Article date
  "id" : 0, # Unique
  "intro" : "Pour sa première édition,...",  # The introduction of the article, as stated on the website
  "title" : "Exposition sur l'alphabet a ka u ku",
  "last_update" : Datetime.datetime.now(),
}
```

**articles_html:** Array of string, with each string:
```
"interesting html content of the page"
```

Method to generate a json from the article:

In [8]:
articleCounter = 0
articles_infos = []
articles_html = []

def generate_articles_json(article_url) :
    global articleCounter
    articleCounter += 1    
    print(articleCounter, ' : ', article_url)
    
    # Getting article
    article_html = requests.get(article_url)
    soup = BeautifulSoup(article_html.content, 'html.parser')
    
    # Content
    content = '<!DOCTYPE html><html lang="fr-FR">{0}<body>{1}</body></html>'.format(soup.head, soup.section)
    
    # Category
    category = soup.find('p', attrs={'class' : 'postmeta'}).text
    
    # Date
    article_date = soup.find('time').attrs.get('datetime')
    
    # Intro
    intro = soup.find('div', attrs={'class' : 'j-module n j-text '}).text
    intro2 = intro.replace("\n   ", "")
    intro3 = intro2[1:len(intro2)-1]
    
    # Title
    title = soup.find('h1', attrs={'class' : 'j-blog-header j-blog-headline'}).text
    
    # Article info
    article_json = {
            'category' : category,
            'date' : article_date,
            'id' : articleCounter,
            'intro' : intro3,
            'title' : title,
            'last_update' : str(datetime.now().date())
        }

    articles_html.append(content)
    articles_infos.append(article_json)

Generate all articles and populates the arrays articles_infos and articles_html

In [9]:
base_url = 'https://www.salonecriture.org'

articleCounter = 0
articles_infos = []
articles_html = []

for article_link in all_links:
    generate_articles_json(base_url + article_link)

1  :  https://www.salonecriture.org/2017/03/02/application-pour-iphone-disponible/
2  :  https://www.salonecriture.org/2017/02/27/sortie-d-une-application-mobile-android-pour-le-salon/
3  :  https://www.salonecriture.org/2017/02/12/ghostwriter-écrivain-fantôme-prête-plume-qu-est-ce/
4  :  https://www.salonecriture.org/2017/01/28/secrétaire-juridique-l-écriture-ce-vecteur-codé-et-procédural/
5  :  https://www.salonecriture.org/2017/01/25/maître-marc-bonnant-à-l-honneur/
6  :  https://www.salonecriture.org/2017/01/17/harmonie-keller-matthey-blogueuse-l-écriture-est-un-art-libérateur/
7  :  https://www.salonecriture.org/2017/01/14/exposition-sur-l-alphabet-a-ka-u-ku/
8  :  https://www.salonecriture.org/2017/01/11/rédacteur-technique-ou-l-art-de-la-vulgarisation/
9  :  https://www.salonecriture.org/2017/01/07/qu-est-ce-qu-un-scriptorium/
10  :  https://www.salonecriture.org/2016/12/20/quels-étaient-les-moyens-de-fabriquer-de-l-encre-noire-au-moyen-âge/
11  :  https://www.salonecriture.org/

## 3. Export data to json files

Filenames:

In [10]:
nowStr = datetime.now().strftime("%Y-%m-%d_%Hh%Mmin%S")

exportPathInfos = os.path.join('export', 'articles_' +nowStr + '_infos.json')
exportPathHtml = os.path.join('export', 'articles_' + nowStr + '_html.json')

Export data:

In [11]:
with open(exportPathInfos, 'w') as f:
        json.dump(articles_infos, f)

In [12]:
with open(exportPathHtml, 'w') as f:
        json.dump(articles_html, f)